# Test Data Preparation Steps

This notebook tests the data preparation logic from `03_data_preparation_blumenstock.ipynb` on a small sample for manual verification.

**Sample:**
- 2 random vintages
- 10 loans per vintage

**Tests:**
1. Behavioral variables (t_act_12m, t_del_30d_12m, t_del_60d_12m)
2. Balance repaid calculation
3. Event determination (prepay, default, censored)
4. Terminal record selection
5. Macroeconomic variable merges

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import column definitions
import sys
sys.path.insert(0, '..')
from src.data.columns import (
    ORIGINATION_COLUMNS, ORIGINATION_DTYPES,
    PERFORMANCE_COLUMNS, PERFORMANCE_DTYPES,
)

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

print("Imports complete.")

Imports complete.


In [2]:
# Configuration
RAW_DATA_DIR = Path('../data/raw')
EXTERNAL_DATA_DIR = Path('../data/external')

# Randomly select 2 vintages from 2010-2025
np.random.seed(42)
ALL_VINTAGES = list(range(2010, 2026))
AVAILABLE_VINTAGES = [v for v in ALL_VINTAGES if list(RAW_DATA_DIR.glob(f'**/sample_{v}'))]

TEST_VINTAGES = np.random.choice(AVAILABLE_VINTAGES, size=min(2, len(AVAILABLE_VINTAGES)), replace=False)
LOANS_PER_VINTAGE = 10
DEFAULT_DELINQUENCY_THRESHOLD = 3

print(f"Available vintages: {len(AVAILABLE_VINTAGES)}")
print(f"Test vintages: {TEST_VINTAGES}")
print(f"Loans per vintage: {LOANS_PER_VINTAGE}")

Available vintages: 16
Test vintages: [2010 2011]
Loans per vintage: 10


## Step 1: Load Sample Data

In [3]:
def load_origination_data(vintage: int) -> pd.DataFrame:
    """Load origination data for a vintage."""
    pattern = f'sample_{vintage}/sample_orig_{vintage}.txt'
    files = list(RAW_DATA_DIR.glob(f'**/{pattern}'))
    if not files:
        return pd.DataFrame()
    return pd.read_csv(
        files[0], sep='|', names=ORIGINATION_COLUMNS,
        dtype=ORIGINATION_DTYPES, na_values=['', ' ']
    )


def load_performance_data(vintage: int) -> pd.DataFrame:
    """Load performance (monthly) data for a vintage."""
    pattern = f'sample_{vintage}/sample_svcg_{vintage}.txt'
    files = list(RAW_DATA_DIR.glob(f'**/{pattern}'))
    if not files:
        return pd.DataFrame()
    return pd.read_csv(
        files[0], sep='|', names=PERFORMANCE_COLUMNS,
        dtype=PERFORMANCE_DTYPES, na_values=['', ' ']
    )


# Load sample for first test vintage
vintage = TEST_VINTAGES[0]
print(f"Loading vintage {vintage}...")

orig_df_full = load_origination_data(vintage)
perf_df_full = load_performance_data(vintage)

print(f"Origination records: {len(orig_df_full):,}")
print(f"Performance records: {len(perf_df_full):,}")

Loading vintage 2010...
Origination records: 50,000
Performance records: 3,414,559


In [4]:
# Sample loans - try to get mix of outcomes
np.random.seed(123)

# Find loans with different outcomes
perf_temp = perf_df_full.copy()
perf_temp['delinquency_status'] = pd.to_numeric(
    perf_temp['current_loan_delinquency_status'].replace({'X': '0', 'XX': '0'}),
    errors='coerce'
).fillna(0).astype(int)

# Find defaulters (90+ days delinquent)
defaulters = perf_temp[perf_temp['delinquency_status'] >= 3]['loan_sequence_number'].unique()

# Find prepayers (zero balance code 01)
prepayers = perf_temp[perf_temp['zero_balance_code'] == '01']['loan_sequence_number'].unique()

# Find censored (neither default nor prepay)
all_loans = perf_temp['loan_sequence_number'].unique()
censored = [l for l in all_loans if l not in defaulters and l not in prepayers]

print(f"Defaulters: {len(defaulters):,}")
print(f"Prepayers: {len(prepayers):,}")
print(f"Censored: {len(censored):,}")

# Select sample with mix
sample_loans = []
n_each = LOANS_PER_VINTAGE // 3

if len(defaulters) >= n_each:
    sample_loans.extend(np.random.choice(defaulters, n_each, replace=False))
elif len(defaulters) > 0:
    sample_loans.extend(defaulters)

if len(prepayers) >= n_each:
    sample_loans.extend(np.random.choice(prepayers, n_each, replace=False))
elif len(prepayers) > 0:
    sample_loans.extend(np.random.choice(prepayers, min(n_each, len(prepayers)), replace=False))

remaining = LOANS_PER_VINTAGE - len(sample_loans)
if len(censored) >= remaining:
    sample_loans.extend(np.random.choice(censored, remaining, replace=False))
elif len(censored) > 0:
    sample_loans.extend(censored[:remaining])


sample_loans = list(set(sample_loans))[:LOANS_PER_VINTAGE]
print(f"\nSelected {len(sample_loans)} sample loans")
print(f"Sample loan IDs: {sample_loans}")

Defaulters: 1,614
Prepayers: 45,518
Censored: 3,517

Selected 10 sample loans
Sample loan IDs: ['F10Q10353237', np.str_('F10Q40490070'), 'F10Q30409134', 'F10Q40317030', np.str_('F10Q10354889'), 'F10Q20235292', 'F10Q10222233', np.str_('F10Q10058142'), np.str_('F10Q30452869'), 'F10Q20193676']


In [5]:
# Filter to sample loans
orig_df = orig_df_full[orig_df_full['loan_sequence_number'].isin(sample_loans)].copy()
perf_df = perf_df_full[perf_df_full['loan_sequence_number'].isin(sample_loans)].copy()

print(f"Sample origination records: {len(orig_df)}")
print(f"Sample performance records: {len(perf_df)}")
print(f"Average months per loan: {len(perf_df) / len(orig_df):.1f}")

Sample origination records: 10
Sample performance records: 1254
Average months per loan: 125.4


In [6]:
# Show origination data
print("=== ORIGINATION DATA ===")
orig_cols = ['loan_sequence_number', 'credit_score', 'orig_ltv', 'orig_dti', 
             'orig_upb', 'orig_interest_rate', 'orig_loan_term', 
             'first_payment_date', 'property_state']
display(orig_df[[c for c in orig_cols if c in orig_df.columns]])

=== ORIGINATION DATA ===


,loan_sequence_number,credit_score,orig_ltv,orig_dti,orig_upb,orig_interest_rate,orig_loan_term,first_payment_date,property_state
1928,F10Q10058142,761,75,27,104000,5.250,360,201003,MN
7624,F10Q10222233,739,80,49,528000,5.375,360,201005,CA
12122,F10Q10353237,757,71,31,79000,5.500,360,201006,MO
12169,F10Q10354889,795,71,22,155000,5.125,360,201005,GA
19062,F10Q20193676,715,59,45,378000,5.250,360,201006,WA
20424,F10Q20235292,758,34,34,386000,4.875,360,201008,NY
35109,F10Q30409134,684,63,38,225000,4.500,180,201010,WI
36194,F10Q30452869,724,68,35,189000,4.250,360,201011,TN
44041,F10Q40317030,774,66,17,79000,4.125,360,201102,KY
47738,F10Q40490070,812,66,13,150000,3.500,180,201101,CO


## Step 2: Parse Performance Data

In [7]:
# Parse reporting period
perf_df['reporting_date'] = pd.to_datetime(
    perf_df['monthly_reporting_period'].astype(str), format='%Y%m'
)
perf_df['year_month'] = perf_df['reporting_date'].dt.to_period('M')

# Parse delinquency status
perf_df['delinquency_status'] = pd.to_numeric(
    perf_df['current_loan_delinquency_status'].replace({'X': '0', 'XX': '0'}),
    errors='coerce'
).fillna(0).astype(int)

# Sort by loan and time
perf_df = perf_df.sort_values(['loan_sequence_number', 'loan_age'])

print("Parsed performance data")
print(f"Date range: {perf_df['reporting_date'].min()} to {perf_df['reporting_date'].max()}")

Parsed performance data
Date range: 2010-02-01 00:00:00 to 2025-06-01 00:00:00


In [8]:
# Show performance history for one loan
test_loan = sample_loans[0]
print(f"=== PERFORMANCE HISTORY: {test_loan} ===")

loan_perf = perf_df[perf_df['loan_sequence_number'] == test_loan][
    ['loan_sequence_number', 'loan_age', 'reporting_date', 
     'current_actual_upb', 'delinquency_status', 'zero_balance_code']
].head(20)

display(loan_perf)

=== PERFORMANCE HISTORY: F10Q10353237 ===


,loan_sequence_number,loan_age,reporting_date,current_actual_upb,delinquency_status,zero_balance_code
786792,F10Q10353237,0,2010-05-01,79000.00,0,NaN
786793,F10Q10353237,1,2010-06-01,79000.00,0,NaN
786794,F10Q10353237,2,2010-07-01,79000.00,0,NaN
786795,F10Q10353237,3,2010-08-01,79000.00,0,NaN
786796,F10Q10353237,4,2010-09-01,79000.00,0,NaN
786797,F10Q10353237,5,2010-10-01,79000.00,0,NaN
786798,F10Q10353237,6,2010-11-01,78000.00,0,NaN
786799,F10Q10353237,7,2010-12-01,78475.16,0,NaN
786800,F10Q10353237,8,2011-01-01,78386.28,0,NaN
786801,F10Q10353237,9,2011-02-01,78141.77,0,NaN


## Step 3: Test Behavioral Variables

Rolling 12-month counts:
- `t_act_12m`: Times NOT delinquent (current) in last 12 months
- `t_del_30d_12m`: Times 30 days delinquent in last 12 months
- `t_del_60d_12m`: Times 60 days delinquent in last 12 months

In [9]:
# Create delinquency indicators
perf_df['is_current'] = (perf_df['delinquency_status'] == 0).astype(int)
perf_df['is_30d_del'] = (perf_df['delinquency_status'] == 1).astype(int)
perf_df['is_60d_del'] = (perf_df['delinquency_status'] == 2).astype(int)

# Rolling counts over last 12 months
grouped = perf_df.groupby('loan_sequence_number')
perf_df['t_act_12m'] = grouped['is_current'].transform(
    lambda x: x.rolling(12, min_periods=1).sum()
)
perf_df['t_del_30d_12m'] = grouped['is_30d_del'].transform(
    lambda x: x.rolling(12, min_periods=1).sum()
)
perf_df['t_del_60d_12m'] = grouped['is_60d_del'].transform(
    lambda x: x.rolling(12, min_periods=1).sum()
)

print("Behavioral variables calculated.")

Behavioral variables calculated.


In [10]:
# MANUAL VERIFICATION: Show behavioral variables for one loan
test_loan = sample_loans[0]
print(f"=== BEHAVIORAL VARIABLES: {test_loan} ===")
print("Verify that rolling sums are calculated correctly over 12-month window.\n")

loan_perf = perf_df[perf_df['loan_sequence_number'] == test_loan][
    ['loan_age', 'delinquency_status', 'is_current', 'is_30d_del', 'is_60d_del',
     't_act_12m', 't_del_30d_12m', 't_del_60d_12m']
].head(24)

display(loan_perf)

print("\nVerification points:")
print("- At month 1: t_act_12m should equal is_current (1 month window)")
print("- At month 12: t_act_12m should equal sum of is_current for months 1-12")
print("- At month 13+: Rolling window of last 12 months only")

=== BEHAVIORAL VARIABLES: F10Q10353237 ===
Verify that rolling sums are calculated correctly over 12-month window.



,loan_age,delinquency_status,is_current,is_30d_del,is_60d_del,t_act_12m,t_del_30d_12m,t_del_60d_12m
786792,0,0,1,0,0,1.0,0.0,0.0
786793,1,0,1,0,0,2.0,0.0,0.0
786794,2,0,1,0,0,3.0,0.0,0.0
786795,3,0,1,0,0,4.0,0.0,0.0
786796,4,0,1,0,0,5.0,0.0,0.0
786797,5,0,1,0,0,6.0,0.0,0.0
786798,6,0,1,0,0,7.0,0.0,0.0
786799,7,0,1,0,0,8.0,0.0,0.0
786800,8,0,1,0,0,9.0,0.0,0.0
786801,9,0,1,0,0,10.0,0.0,0.0



Verification points:
- At month 1: t_act_12m should equal is_current (1 month window)
- At month 12: t_act_12m should equal sum of is_current for months 1-12
- At month 13+: Rolling window of last 12 months only


In [11]:
# Manual spot check for month 12
test_loan = sample_loans[0]
loan_data = perf_df[perf_df['loan_sequence_number'] == test_loan].reset_index(drop=True)

if len(loan_data) >= 12:
    # Check month 12 (index 11)
    expected_t_act_12m = loan_data.iloc[0:12]['is_current'].sum()
    actual_t_act_12m = loan_data.iloc[11]['t_act_12m']
    
    print(f"Month 12 check for loan {test_loan}:")
    print(f"  Sum of is_current for months 1-12: {expected_t_act_12m}")
    print(f"  t_act_12m at month 12: {actual_t_act_12m}")
    print(f"  Match: {expected_t_act_12m == actual_t_act_12m} ✓" if expected_t_act_12m == actual_t_act_12m else f"  MISMATCH ✗")
else:
    print(f"Loan {test_loan} has fewer than 12 months of data")

Month 12 check for loan F10Q10353237:
  Sum of is_current for months 1-12: 12
  t_act_12m at month 12: 12.0
  Match: True ✓


## Step 4: Test Balance Repaid Calculation

`bal_repaid = (orig_upb - current_upb) / orig_upb * 100`

In [12]:
# Get original UPB from origination data
orig_upb = orig_df.set_index('loan_sequence_number')['orig_upb']
perf_df['orig_upb_lookup'] = perf_df['loan_sequence_number'].map(orig_upb)

# Calculate balance repaid
perf_df['bal_repaid'] = (
    (perf_df['orig_upb_lookup'] - perf_df['current_actual_upb'].fillna(0)) / 
    perf_df['orig_upb_lookup']
) * 100
perf_df['bal_repaid'] = perf_df['bal_repaid'].clip(0, 100)

print("Balance repaid calculated.")

Balance repaid calculated.


In [13]:
# MANUAL VERIFICATION: Balance repaid
test_loan = sample_loans[0]
print(f"=== BALANCE REPAID: {test_loan} ===")

# Get original UPB
loan_orig_upb = orig_df[orig_df['loan_sequence_number'] == test_loan]['orig_upb'].iloc[0]
print(f"Original UPB: ${loan_orig_upb:,.2f}")

loan_perf = perf_df[perf_df['loan_sequence_number'] == test_loan][
    ['loan_age', 'current_actual_upb', 'orig_upb_lookup', 'bal_repaid']
]

print("\nFirst 10 and last 5 months:")
display(pd.concat([loan_perf.head(10), loan_perf.tail(5)]))

# Manual calculation for first month
first_row = loan_perf.iloc[0]
manual_bal_repaid = (first_row['orig_upb_lookup'] - first_row['current_actual_upb']) / first_row['orig_upb_lookup'] * 100
print(f"\nManual check for month 1:")
print(f"  (${first_row['orig_upb_lookup']:,.2f} - ${first_row['current_actual_upb']:,.2f}) / ${first_row['orig_upb_lookup']:,.2f} * 100")
print(f"  = {manual_bal_repaid:.4f}%")
print(f"  bal_repaid column: {first_row['bal_repaid']:.4f}%")
print(f"  Match: {abs(manual_bal_repaid - first_row['bal_repaid']) < 0.0001} ✓" if abs(manual_bal_repaid - first_row['bal_repaid']) < 0.0001 else "  MISMATCH ✗")

=== BALANCE REPAID: F10Q10353237 ===
Original UPB: $79,000.00

First 10 and last 5 months:


,loan_age,current_actual_upb,orig_upb_lookup,bal_repaid
786792,0,79000.00,79000,0.0
786793,1,79000.00,79000,0.0
786794,2,79000.00,79000,0.0
786795,3,79000.00,79000,0.0
786796,4,79000.00,79000,0.0
786797,5,79000.00,79000,0.0
786798,6,78000.00,79000,1.265823
786799,7,78475.16,79000,0.664354
786800,8,78386.28,79000,0.776861
786801,9,78141.77,79000,1.086367



Manual check for month 1:
  ($79,000.00 - $79,000.00) / $79,000.00 * 100
  = 0.0000%
  bal_repaid column: 0.0000%
  Match: True ✓


## Step 5: Test Event Determination

Event definitions:
- **Default (k=2)**: First time reaching 90+ days delinquent (delinquency_status >= 3)
- **Prepayment (k=1)**: Zero balance code = '01' (voluntary prepayment)
- **Censored (k=0)**: Neither default nor prepayment

In [14]:
# Default: first time reaching 90+ days delinquent
perf_df['is_default'] = (perf_df['delinquency_status'] >= DEFAULT_DELINQUENCY_THRESHOLD).astype(int)
perf_df['first_default'] = grouped['is_default'].transform(
    lambda x: (x.cumsum() == 1) & (x == 1)
).astype(int)

# Prepayment: zero balance code = '01'
perf_df['is_prepay'] = (perf_df['zero_balance_code'] == '01').astype(int)

print("Event indicators calculated.")
print(f"Loans with default event: {perf_df.groupby('loan_sequence_number')['first_default'].max().sum()}")
print(f"Loans with prepay event: {perf_df.groupby('loan_sequence_number')['is_prepay'].max().sum()}")

Event indicators calculated.
Loans with default event: 3
Loans with prepay event: 5


In [15]:
# MANUAL VERIFICATION: Show events for all sample loans
print("=== EVENT SUMMARY FOR ALL SAMPLE LOANS ===")

for loan in sample_loans:
    loan_data = perf_df[perf_df['loan_sequence_number'] == loan]
    max_del = loan_data['delinquency_status'].max()
    has_default = loan_data['first_default'].max()
    has_prepay = loan_data['is_prepay'].max()
    zb_codes = loan_data['zero_balance_code'].dropna().unique()
    duration = loan_data['loan_age'].max()
    
    event = 'DEFAULT' if has_default else ('PREPAY' if has_prepay else 'CENSORED')
    
    print(f"\nLoan {loan}:")
    print(f"  Duration: {duration} months")
    print(f"  Max delinquency: {max_del} months")
    print(f"  Zero balance codes: {zb_codes}")
    print(f"  Event: {event}")

=== EVENT SUMMARY FOR ALL SAMPLE LOANS ===

Loan F10Q10353237:
  Duration: 175 months
  Max delinquency: 34 months
  Zero balance codes: ['01']
  Event: DEFAULT

Loan F10Q40490070:
  Duration: 174 months
  Max delinquency: 0 months
  Zero balance codes: []
  Event: CENSORED

Loan F10Q30409134:
  Duration: 13 months
  Max delinquency: 0 months
  Zero balance codes: ['01']
  Event: PREPAY

Loan F10Q40317030:
  Duration: 94 months
  Max delinquency: 0 months
  Zero balance codes: ['01']
  Event: PREPAY

Loan F10Q10354889:
  Duration: 182 months
  Max delinquency: 0 months
  Zero balance codes: []
  Event: CENSORED

Loan F10Q20235292:
  Duration: 179 months
  Max delinquency: 11 months
  Zero balance codes: []
  Event: DEFAULT

Loan F10Q10222233:
  Duration: 11 months
  Max delinquency: 0 months
  Zero balance codes: ['01']
  Event: PREPAY

Loan F10Q10058142:
  Duration: 184 months
  Max delinquency: 0 months
  Zero balance codes: []
  Event: CENSORED

Loan F10Q30452869:
  Duration: 176 mo

In [16]:
# Show detailed history for a default loan (if exists)
default_loans = [l for l in sample_loans 
                 if perf_df[perf_df['loan_sequence_number'] == l]['first_default'].max() == 1]

if default_loans:
    test_loan = default_loans[0]
    print(f"=== DEFAULT LOAN HISTORY: {test_loan} ===")
    print("Look for first month where delinquency_status >= 3")
    
    loan_perf = perf_df[perf_df['loan_sequence_number'] == test_loan][
        ['loan_age', 'delinquency_status', 'is_default', 'first_default', 'zero_balance_code']
    ]
    display(loan_perf)
else:
    print("No default loans in sample")

=== DEFAULT LOAN HISTORY: F10Q10353237 ===
Look for first month where delinquency_status >= 3


,loan_age,delinquency_status,is_default,first_default,zero_balance_code
786792,0,0,0,0,NaN
786793,1,0,0,0,NaN
786794,2,0,0,0,NaN
786795,3,0,0,0,NaN
786796,4,0,0,0,NaN
...,...,...,...,...,...
786963,171,2,0,0,NaN
786964,172,3,1,0,NaN
786965,173,1,0,0,NaN
786966,174,2,0,0,NaN


In [17]:
# Show detailed history for a prepay loan (if exists)
prepay_loans = [l for l in sample_loans 
                if perf_df[perf_df['loan_sequence_number'] == l]['is_prepay'].max() == 1]

if prepay_loans:
    test_loan = prepay_loans[0]
    print(f"=== PREPAY LOAN HISTORY: {test_loan} ===")
    print("Look for zero_balance_code = '01'")
    
    loan_perf = perf_df[perf_df['loan_sequence_number'] == test_loan][
        ['loan_age', 'delinquency_status', 'current_actual_upb', 'zero_balance_code', 'is_prepay']
    ]
    display(loan_perf.tail(15))
else:
    print("No prepay loans in sample")

=== PREPAY LOAN HISTORY: F10Q10353237 ===
Look for zero_balance_code = '01'


,loan_age,delinquency_status,current_actual_upb,zero_balance_code,is_prepay
786953,161,2,59845.49,NaN,0
786954,162,3,59845.49,NaN,0
786955,163,4,59845.49,NaN,0
786956,164,2,59295.50,NaN,0
786957,165,3,59295.50,NaN,0
786958,166,1,58737.92,NaN,0
786959,167,2,58737.92,NaN,0
786960,168,3,58737.92,NaN,0
786961,169,2,58361.92,NaN,0
786962,170,1,57982.47,NaN,0


## Step 6: Test Terminal Record Selection

Logic:
1. If default occurred: terminal record = first default month
2. Else if prepay occurred: terminal record = prepay month
3. Else: terminal record = last observed month (censored)

In [24]:
def get_terminal_event(group):
    """Get terminal record with event type."""
    last_row = group.iloc[-1].copy()
    
    # Check for default (first 90+ delinquency)
    default_rows = group[group['first_default'] == 1]
    if len(default_rows) > 0:
        last_row = default_rows.iloc[0].copy()
        last_row['event_code'] = 2  # Default
        return last_row
    
    # Check for prepayment
    prepay_rows = group[group['is_prepay'] == 1]
    if len(prepay_rows) > 0:
        last_row = prepay_rows.iloc[0].copy()
        last_row['event_code'] = 1  # Prepay
        return last_row
    
    # Censored
    last_row['event_code'] = 0
    return last_row


# Get terminal records
terminal_df = perf_df.groupby('loan_sequence_number').apply(get_terminal_event)
terminal_df = terminal_df.reset_index(drop=True)

print(f"Terminal records: {len(terminal_df)}")
print(f"\nEvent distribution:")
print(terminal_df['event_code'].value_counts().sort_index())

Terminal records: 10

Event distribution:
event_code
0    4
1    3
2    3
Name: count, dtype: int64


In [25]:
# MANUAL VERIFICATION: Show terminal records
print("=== TERMINAL RECORDS ===")
display(terminal_df[[
    'loan_sequence_number', 'loan_age', 'delinquency_status', 
    'zero_balance_code', 'event_code', 'bal_repaid',
    't_act_12m', 't_del_30d_12m', 't_del_60d_12m'
]])

print("\nEvent codes: 0=Censored, 1=Prepay, 2=Default")

=== TERMINAL RECORDS ===


,loan_sequence_number,loan_age,delinquency_status,zero_balance_code,event_code,bal_repaid,t_act_12m,t_del_30d_12m,t_del_60d_12m
0,F10Q10058142,184,0,NaN,0,32.646019,12.0,0.0,0.0
1,F10Q10222233,11,0,01,1,100.000000,12.0,0.0,0.0
2,F10Q10353237,100,3,NaN,2,11.803873,1.0,0.0,10.0
3,F10Q10354889,182,0,NaN,0,46.305103,12.0,0.0,0.0
4,F10Q20193676,57,3,NaN,2,6.972712,9.0,1.0,1.0
5,F10Q20235292,130,3,NaN,2,20.284575,9.0,1.0,1.0
6,F10Q30409134,13,0,01,1,100.000000,12.0,0.0,0.0
7,F10Q30452869,176,0,NaN,0,34.488693,12.0,0.0,0.0
8,F10Q40317030,94,0,01,1,100.000000,12.0,0.0,0.0
9,F10Q40490070,174,0,NaN,0,95.753620,12.0,0.0,0.0



Event codes: 0=Censored, 1=Prepay, 2=Default


## Step 7: Test Macro Variable Merge

Merge macroeconomic data at the observation time (terminal record).

In [26]:
# Load macro data
macro_path = EXTERNAL_DATA_DIR / 'fred_monthly_panel.parquet'
if macro_path.exists():
    macro_national = pd.read_parquet(macro_path)
    macro_national.index.name = 'date'
    macro_national = macro_national.reset_index()
    macro_national['date'] = pd.to_datetime(macro_national['date'])
    macro_national['year_month'] = macro_national['date'].dt.to_period('M')
    print(f"Macro data loaded: {len(macro_national)} months")
    print(f"Columns: {list(macro_national.columns)[:10]}...")
else:
    print("Macro data not found at", macro_path)
    macro_national = None

Macro data loaded: 337 months
Columns: ['date', 'UNRATE', 'MORTGAGE30US', 'CSUSHPINSA', 'USSTHPI', 'FEDFUNDS', 'T10Y2Y', 'DGS3MO', 'DGS10', 'DGS5']...


In [27]:
if macro_national is not None:
    # Merge macro data
    macro_cols = ['year_month', 'MORTGAGE30US', 'DGS10']
    macro_cols = [c for c in macro_cols if c in macro_national.columns]
    
    terminal_with_macro = terminal_df.merge(
        macro_national[macro_cols],
        on='year_month',
        how='left'
    )
    
    print("=== TERMINAL RECORDS WITH MACRO ===")
    display(terminal_with_macro[[
        'loan_sequence_number', 'loan_age', 'year_month', 'event_code',
        'MORTGAGE30US', 'DGS10'
    ] if 'MORTGAGE30US' in terminal_with_macro.columns else [
        'loan_sequence_number', 'loan_age', 'year_month', 'event_code'
    ]])
    
    # Check coverage
    if 'MORTGAGE30US' in terminal_with_macro.columns:
        coverage = terminal_with_macro['MORTGAGE30US'].notna().mean()
        print(f"\nMacro coverage: {coverage:.1%}")
else:
    print("Skipping macro merge - data not available")

=== TERMINAL RECORDS WITH MACRO ===


,loan_sequence_number,loan_age,year_month,event_code,MORTGAGE30US,DGS10
0,F10Q10058142,184,2025-06,0,6.8175,4.383500
1,F10Q10222233,11,2011-03,1,4.8360,3.414348
2,F10Q10353237,100,2018-09,2,4.6275,3.004211
3,F10Q10354889,182,2025-06,0,6.8175,4.383500
4,F10Q20193676,57,2015-02,2,3.7100,1.975263
5,F10Q20235292,130,2021-05,2,2.9625,1.621000
6,F10Q30409134,13,2011-10,1,4.0675,2.152000
7,F10Q30452869,176,2025-06,0,6.8175,4.383500
8,F10Q40317030,94,2018-11,1,4.8660,3.117000
9,F10Q40490070,174,2025-06,0,6.8175,4.383500



Macro coverage: 100.0%


## Summary: Manual Verification Checklist

Use this checklist to verify the data preparation is working correctly:

In [28]:
print("=" * 60)
print("DATA PREPARATION VERIFICATION CHECKLIST")
print("=" * 60)

print("\n1. BEHAVIORAL VARIABLES")
print("   [ ] t_act_12m: Count of current months in rolling 12-month window")
print("   [ ] t_del_30d_12m: Count of 30-day delinquent months in window")
print("   [ ] t_del_60d_12m: Count of 60-day delinquent months in window")
print("   [ ] At month 12, sum equals sum of first 12 months")
print("   [ ] At month 13+, window properly slides")

print("\n2. BALANCE REPAID")
print("   [ ] Formula: (orig_upb - current_upb) / orig_upb * 100")
print("   [ ] Values clipped to [0, 100]")
print("   [ ] Increases over time as loan is paid down")

print("\n3. EVENT DETERMINATION")
print("   [ ] Default: First time delinquency_status >= 3")
print("   [ ] Prepay: zero_balance_code = '01'")
print("   [ ] Censored: Neither default nor prepay")

print("\n4. TERMINAL RECORD")
print("   [ ] Default loans: Record at first 90+ day delinquency")
print("   [ ] Prepay loans: Record at prepayment month")
print("   [ ] Censored loans: Last observed month")
print("   [ ] Duration (loan_age) matches event timing")

print("\n5. MACRO MERGE")
print("   [ ] Macro data merged on year_month")
print("   [ ] No unexpected nulls after merge")

print("\n" + "=" * 60)

DATA PREPARATION VERIFICATION CHECKLIST

1. BEHAVIORAL VARIABLES
   [ ] t_act_12m: Count of current months in rolling 12-month window
   [ ] t_del_30d_12m: Count of 30-day delinquent months in window
   [ ] t_del_60d_12m: Count of 60-day delinquent months in window
   [ ] At month 12, sum equals sum of first 12 months
   [ ] At month 13+, window properly slides

2. BALANCE REPAID
   [ ] Formula: (orig_upb - current_upb) / orig_upb * 100
   [ ] Values clipped to [0, 100]
   [ ] Increases over time as loan is paid down

3. EVENT DETERMINATION
   [ ] Default: First time delinquency_status >= 3
   [ ] Prepay: zero_balance_code = '01'
   [ ] Censored: Neither default nor prepay

4. TERMINAL RECORD
   [ ] Default loans: Record at first 90+ day delinquency
   [ ] Prepay loans: Record at prepayment month
   [ ] Censored loans: Last observed month
   [ ] Duration (loan_age) matches event timing

5. MACRO MERGE
   [ ] Macro data merged on year_month
   [ ] No unexpected nulls after merge



## Run with Different Vintages

To test with different data, change `np.random.seed()` in the config cell and re-run the notebook.